In [ ]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl

from util import *
import var
import cut
import data
import hist

import importlib

from pyanalib import panda_helpers

In [ ]:
importlib.reload(var)
importlib.reload(cut)
importlib.reload(data)
importlib.reload(hist)
importlib.reload(panda_helpers)

In [ ]:
mpl.rc('font', size=14)

dosave = True
savedir = "./plots/mc2d_varytrue_"

import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
filedir = "/icarus/data/users/gputnam/DP2022P/reco/"

onbeamfile = filedir + "majorityH4_trkdf.df"
offbeamfile = filedir + "offbeam_majorityH4_trk.df"
mcfile = filedir + "mcnu_trk.df"

In [ ]:
mc = data.mc_dataset(mcfile, "trk")
mc.df

In [ ]:
mchitdf = pd.read_hdf(mcfile, "hit")
mchitdf

In [ ]:
onbeam = data.onbeam_dataset(onbeamfile, "trk")
onbeam.df

In [ ]:
offbeam = data.offbeam_dataset(offbeamfile, "trk")
offbeam.df

In [ ]:
importlib.reload(var)
importlib.reload(cut)
importlib.reload(data)
importlib.reload(hist)

In [ ]:
POTSTR = "$%.2f \\times 10^{18}$ POT" % (onbeam.POT/1e18)

In [ ]:
bins = np.linspace(-1, 1, 21)

_ = hist.plotmc(plt, var.crlongtrkdiry@var.slc, mc, cut=cut.fiducial@var.slc, 
                bins=bins, POT=onbeam.POT, fluxsyst=True, geniesyst=True,
               hist_kw={"histtype": "step"})
_ = hist.plotdata(plt, var.crlongtrkdiry, onbeam.slc, offbeam.slc, cut=cut.fiducial, bins=bins)

In [ ]:
chi2p_bins = np.linspace(0, 400, 41)
chi2u_bins = np.linspace(0, 80, 41)

basecut = cut.fiducial@var.slc & (var.TRKVAR.len > 5)

@var.VAR
def ntrk(df):
    return df.groupby(level=[0,1,2]).size()

cuts = [
    basecut,
    basecut & (var.crlongtrkdiry@var.slc > -0.4) & (ntrk >= 3),
    basecut & (var.crlongtrkdiry@var.slc > -0.4) & (ntrk >= 3) & (var.TRKVAR.len > 100)
]

titles = [
    "Fiducial Tracks",
    "Fiducial, Cos. Rej. Tracks",
    "Fiducial, Cos. Rej., Long Tracks"
]

labels = ["fid", "fid_cosrej", "fid_cosrej_long"]

In [ ]:
v = var.chi2p@var.trk

for i, c in enumerate(cuts):
    plt.figure(3*i)

    _ = hist.plotmc(plt, v, mc, cut=c, bins=chi2p_bins, POT=onbeam.POT, fluxsyst=True, geniesyst=True,
                   hist_kw={"histtype": "step"})
    _ = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=chi2p_bins)

    plt.xlabel("$\\chi^2_p$")
    plt.ylabel("Tracks / %s" % POTSTR)
    plt.title(titles[i])
    
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "chi2p_%s.pdf" % labels[i])
    
    plt.figure(3*i+1)
    
    _ = hist.plotmc(plt, v, mc, cut=c, bins=chi2p_bins, areanorm=True)
    _ = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=chi2p_bins, areanorm=True)

    plt.xlabel("$\\chi^2_p$")
    plt.ylabel("Area Normalized")
    plt.title(titles[i])
    
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "chi2p_areanorm_%s.pdf" % labels[i])
    
    plt.figure(3*i+2)
    _,_,l = hist.plotmc(plt, v, mc, cut=c, bins=chi2p_bins, POT=onbeam.POT, fluxsyst=True, geniesyst=True,
                    categories=map(lambda f: f@var.trk, cut.trk_mc_categories),
                   hist_kw={"stacked": True, "label": cut.trk_mc_labels})
    _ = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=chi2p_bins)
    plt.legend(l, cut.trk_mc_labels, ncol=2)
    plt.xlabel("$\\chi^2_p$")
    plt.ylabel("Tracks / %s" % POTSTR)
    plt.title(titles[i])
    
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "chi2p_categories_%s.pdf" % labels[i])

In [ ]:
v = var.chi2u@var.trk

for i, c in enumerate(cuts):
    plt.figure(3*i)

    _ = hist.plotmc(plt, v, mc, cut=c, bins=chi2u_bins, POT=onbeam.POT, fluxsyst=True, geniesyst=True,
                   hist_kw={"histtype": "step"})
    _ = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=chi2u_bins)

    plt.xlabel("$\\chi^2_\\mu$")
    plt.ylabel("Tracks / %s" % POTSTR)
    plt.title(titles[i])
    
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "chi2u_%s.pdf" % labels[i])
    
    plt.figure(3*i+1)
    
    _ = hist.plotmc(plt, v, mc, cut=c, bins=chi2u_bins, areanorm=True)
    _ = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=chi2u_bins, areanorm=True)

    plt.xlabel("$\\chi^2_\\mu$")
    plt.ylabel("Area Normalized")
    plt.title(titles[i])
    
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "chi2u_areanorm_%s.pdf" % labels[i])
    
    plt.figure(3*i+2)
    _,_,l = hist.plotmc(plt, v, mc, cut=c, bins=chi2u_bins, POT=onbeam.POT, fluxsyst=True, geniesyst=True,
                    categories=map(lambda f: f@var.trk, cut.trk_mc_categories),
                   hist_kw={"stacked": True, "label": cut.trk_mc_labels})
    _ = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=chi2u_bins)
    plt.legend(l, cut.trk_mc_labels, ncol=2)
    plt.xlabel("$\\chi^2_\\mu$")
    plt.ylabel("Tracks / %s" % POTSTR)
    plt.title(titles[i])
    
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "chi2u_categories_%s.pdf" % labels[i])

In [ ]:
# # Vary the calibration

# mchitdf["dqdx_e"] = mchitdf.dqdx*np.exp((mchitdf.t-850)*0.4/3e3)/0.01235

# # Try out systematic variations on chi2
# dedx_loA = var.dedxdf(dqdxname="dqdx_e", A=0.91)(mchitdf)
# dedx_hiA = var.dedxdf(dqdxname="dqdx_e", A=0.95)(mchitdf)

# dedx_loB = var.dedxdf(dqdxname="dqdx_e", B=0.212*0.95)(mchitdf)
# dedx_hiB = var.dedxdf(dqdxname="dqdx_e", B=0.212/0.95)(mchitdf)

In [ ]:
# Vary the truth

mchitdf["dqdx_e"] = mchitdf.dqdx*np.exp((mchitdf.t-850)*0.4/3e3)/0.01235

mchitdf["dqdx_e_loA"] = mchitdf.dqdx_e * var.scale_recombination(A=0.91)(mchitdf.truth.e / mchitdf.pitch)
mchitdf["dqdx_e_hiA"] = mchitdf.dqdx_e * var.scale_recombination(A=0.95)(mchitdf.truth.e / mchitdf.pitch)

# Try out systematic variations on recombination / gain
dedx_loA = var.dedxdf(dqdxname="dqdx_e_loA")(mchitdf)
dedx_hiA = var.dedxdf(dqdxname="dqdx_e_hiA")(mchitdf)

mchitdf["dqdx_e_loB"] = mchitdf.dqdx_e * var.scale_recombination(B=0.212*0.95)(mchitdf.truth.e / mchitdf.pitch)
mchitdf["dqdx_e_hiB"] = mchitdf.dqdx_e * var.scale_recombination(B=0.212*1.05)(mchitdf.truth.e / mchitdf.pitch)

# Try out systematic variations on chi2
dedx_loA = var.dedxdf(dqdxname="dqdx_e_loA")(mchitdf)
dedx_hiA = var.dedxdf(dqdxname="dqdx_e_hiA")(mchitdf)

dedx_loB = var.dedxdf(dqdxname="dqdx_e_loB")(mchitdf)
dedx_hiB = var.dedxdf(dqdxname="dqdx_e_hiB")(mchitdf)

In [ ]:
if "chi2u_loA" in mc.df: del mc.df["chi2u_loA"]
if "chi2p_loA" in mc.df: del mc.df["chi2p_loA"]
if "chi2u_hiA" in mc.df: del mc.df["chi2u_hiA"]
if "chi2p_hiA" in mc.df: del mc.df["chi2p_hiA"]

mc.df = panda_helpers.multicol_add(mc.df, var.hchi2u(dedx_loA).rename("chi2u_loA"))
mc.df = panda_helpers.multicol_add(mc.df, var.hchi2p(dedx_loA).rename("chi2p_loA"))
mc.df = panda_helpers.multicol_add(mc.df, var.hchi2u(dedx_hiA).rename("chi2u_hiA"))
mc.df = panda_helpers.multicol_add(mc.df, var.hchi2p(dedx_hiA).rename("chi2p_hiA"))

In [ ]:
if "chi2u_loB" in mc.df: del mc.df["chi2u_loB"]
if "chi2p_loB" in mc.df: del mc.df["chi2p_loB"]
if "chi2u_hiB" in mc.df: del mc.df["chi2u_hiB"]
if "chi2p_hiB" in mc.df: del mc.df["chi2p_hiB"]

mc.df = panda_helpers.multicol_add(mc.df, var.hchi2u(dedx_loB).rename("chi2u_loB"))
mc.df = panda_helpers.multicol_add(mc.df, var.hchi2p(dedx_loB).rename("chi2p_loB"))
mc.df = panda_helpers.multicol_add(mc.df, var.hchi2u(dedx_hiB).rename("chi2u_hiB"))
mc.df = panda_helpers.multicol_add(mc.df, var.hchi2p(dedx_hiB).rename("chi2p_hiB"))

In [ ]:
v = var.chi2u@var.trk
c = basecut & (var.crlongtrkdiry@var.slc > -0.4) & (ntrk >= 3) & (v > 0)

_, _, h1 = hist.plotmc(plt, v, mc, cut=c, bins=chi2u_bins, areanorm=True, 
                hist_kw={"histtype": "step"})
_, _, h2 = hist.plotmc(plt, var.DF.chi2u_loA, mc, cut=c, bins=chi2u_bins, areanorm=True, 
                hist_kw={"histtype": "step"})
_, _, h3 = hist.plotmc(plt, var.DF.chi2u_hiA, mc, cut=c, bins=chi2u_bins, areanorm=True, 
                hist_kw={"histtype": "step"})

_ = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=chi2u_bins, areanorm=True)

plt.legend(handles = [h1[0], h2[0], h3[0]], labels=["Nominal", "Low $\\alpha$", "High $\\alpha$"])
plt.xlabel("$\\chi^2_\\mu$")
plt.ylabel("Area Normalized")

if dosave: plt.savefig(savedir + "areanorm_cosmicrej_chi2u_varyA.pdf")

In [ ]:
v = var.chi2u@var.trk
c = basecut & (var.crlongtrkdiry@var.slc > -0.4) & (ntrk >= 3) & (v > 0)

_, _, h1 = hist.plotmc(plt, v, mc, cut=c, bins=chi2u_bins, areanorm=True, 
                hist_kw={"histtype": "step"})
_, _, h2 = hist.plotmc(plt, var.DF.chi2u_loB, mc, cut=c, bins=chi2u_bins, areanorm=True, 
                hist_kw={"histtype": "step"})
_, _, h3 = hist.plotmc(plt, var.DF.chi2u_hiB, mc, cut=c, bins=chi2u_bins, areanorm=True, 
                hist_kw={"histtype": "step"})

_ = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=chi2u_bins, areanorm=True)

plt.legend(handles = [h1[0], h2[0], h3[0]], labels=["Nominal", "Low $\\beta$", "High $\\beta$"])
plt.xlabel("$\\chi^2_\\mu$")
plt.ylabel("Area Normalized")

if dosave: plt.savefig(savedir + "areanorm_cosmicrej_chi2u_varyB.pdf")

In [ ]:
bins = np.linspace(0, 1, 21)
c = cut.fiducial@var.slc & cut.is_proton@var.trk

v = var.trkscore@var.pfp

_ = hist.plotmc(plt, v, mc, cut=c, bins=bins, POT=onbeam.POT, fluxsyst=True)
_ = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=bins)

plt.ylabel("Proton-Like Particles / %s" % POTSTR)
plt.xlabel("Pandora Track Score")

plt.tight_layout()
if dosave: plt.savefig(savedir + "proton_trkscore.pdf")

In [ ]:
bins = np.linspace(0, 1, 21)
c = cut.fiducial@var.slc & cut.is_proton@var.trk

v = var.trkscore@var.pfp

_ = hist.plotmc(plt, v, mc, cut=c, bins=bins, areanorm=True)
_ = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=bins, areanorm=True)

In [ ]:
bins = np.linspace(0, 1, 21)
c = cut.fiducial@var.slc & cut.is_proton@var.trk

v = var.chgendfrac@var.pfp

_ = hist.plotmc(plt, v, mc, cut=c, bins=bins, POT=onbeam.POT, fluxsyst=True)
_ = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=bins)

In [ ]:
bins = np.linspace(0, 1, 21)
c = cut.fiducial@var.slc & cut.is_proton@var.trk

v = var.chgfracspread@var.pfp

_ = hist.plotmc(plt, v, mc, cut=c, bins=bins, POT=onbeam.POT)
_ = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=bins)

In [ ]:
bins = np.linspace(0, 0.05, 21)
c = cut.fiducial@var.slc & cut.is_proton@var.trk

v = var.linfitdiff@var.pfp

_ = hist.plotmc(plt, v, mc, cut=c, bins=bins, POT=onbeam.POT)
_ = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=bins)